# Bereiten wir das Script vor

In [4]:
#Imports
import os #Files in listen abgreifen
import tqdm #anzeigen, wo wir stehen
import textract
import pandas as pd


## Vorbereitung

In [5]:
#Als Vorbereitung, machen aus dem Inhalt des Folders eine Liste. Es handelt sich hierbei
#um eine Liste aller pdfs im Ordner pdfs. Es sind die Swissmedic Recalls (#ImplantFiles)
pdfs = os.listdir('pdfs')

## Fortschritt anzeigen

In [6]:
from tqdm import tqdm
for i in tqdm(range(1000000)): #100000000000
    elem = i

100%|██████████| 1000000/1000000 [00:00<00:00, 2481228.86it/s]


## Ein File auslesen

In [7]:
#hier lesen wir in PDF aus. .process kommt am häufigesten vor.
text = textract.process("pdfs/"+pdfs[0], method='pdfminer', encoding='ISO-8859-1') #method='pdfminer', encoding='ISO-8859-1'
# der text wird ausgelesen mit textract.process

In [8]:
print("pdfs/"+pdfs[0]) #hier sehen wir das Dokument, das wir lokal gespeichert haben. 

pdfs/Vk_20180514_49documents0.pdf


In [9]:
str(text) #so bauen wir aus den Daten einen String.

"b'\\x0c\\x0c\\x0c'"

## Alle Files auslesen

In [10]:
lst = []
mistakes = []
for pdf in tqdm(pdfs): # wir packen die PDF in die tqdm, damit wir sehen, wo wir sind (tqdm ist ein Anzeigetool)
    try:
        text = textract.process("pdfs/"+pdf, method='pdfminer', encoding='ISO-8859-1')
        lst.append(text)
    except:
        mistakes.append(pdf)
        
# methode: method='pdfminer'
# wir bauen eine liste und wir bauen eine liste mit mistakes

100%|██████████| 82/82 [02:06<00:00,  1.06it/s]


In [11]:
len(lst)

81

In [12]:
mistakes

['Vk_20180517_12documents0.pdf']

## Bilder auslesen

In [13]:
text = textract.process("pdfs/"+pdfs[0], method='tesseract') #Nicht ideal, aber gut

## Alles zusammen

In [14]:
#Rohe Analyse
for pdf in tqdm(pdfs):
    try:
        text = textract.process("pdfs/"+pdf, method='pdfminer', encoding='ISO-8859-1')
        text = text.decode("ISO-8859-1").replace("\n", " ")
        if len(text) < 40: # text mit weniger als 40 Zeichen könnte ein Bild zeigen, deshalb als Bild anzeigen.
            text = textract.process("pdfs/"+pdf, method='tesseract', language='deu')
            text = text.decode("ISO-8859-1").replace("\n", " ")
            
        mini_dict = {'Text':text, # wir bauen eine liste von mini-dicts
                     'File':pdf}
        lst.append(mini_dict)
    except:
        mini_dict = {'Text':'Fehlermeldung',
                     'File':pdf}
        lst.append(mini_dict)

100%|██████████| 82/82 [02:13<00:00,  1.14it/s]


In [15]:
#Erste Säuberung
df = pd.DataFrame(lst)
def date(elem):
    elem = elem[3:11]
    return elem
df['date'] = df['File'].apply(date)
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df.index = df['date']
#Suchen wir noch nach Implantaten
df['implant'] = df['Text'].str.contains('implant')
df.to_csv('datafile.csv')

KeyError: 'File'